In [1]:
%load_ext rpy2.ipython

# Имитация отжига (Процедура локального поиска)

In [2]:
%%R

hchange=function(par,lower,upper,dist,round=TRUE,...)
{ 
  D=length(par) 
  step=dist(D,...)
  if(round) 
    step=round(step)
  par1=par+step
  return(ifelse(par1<lower,lower,ifelse(par1>upper,upper,par1)))
}

bchange=function(par)
{ 
  D=length(par)
  hchange(par,lower=rep(0,D),upper=rep(1,D),rnorm,mean=0,sd=1)
}
D=8

# Методы Монте-Карло

In [6]:
%%R

mcsearch=function(N,lower,upper,FUN,type="min",...)
{ 
  D=length(lower)
  s=matrix(nrow=N,ncol=D)
  for(i in 1:N) 
    s[i,]=runif(D,lower,upper)
  fsearch(s,FUN,type,...)
}

fsearch=function(search,FUN,type="min",...)
{
    x=apply(search,1,FUN,...)
    ib=switch(type,min=which.min(x),max=which.max(x))
    return(list(index=ib,sol=search[ib,],eval=x[ib]))
}

# Восхождение к вершине (Процедура локального поиска)

In [12]:
%%R

hclimbing=function(par,fn,change,lower,upper,control, type="min",...)
{ 
    fpar=fn(par,...)
    for(i in 1:control$maxit)
    {
      par1=change(par,lower,upper)
      fpar1=fn(par1,...)
      if(control$REPORT>0 &&(i==1||i%%control$REPORT==0))
        cat("i:",i,"s:",par,"f:",fpar,"s’",par1,"f:",fpar1,"\n")
      if( (type=="min" && fpar1<fpar) || (type=="max" && fpar1>fpar)) 
      { 
        par=par1;fpar=fpar1 
      }
    }
    if(control$REPORT>=1) cat("best:",par,"f:",fpar,"\n")
      return(list(sol=par,eval=fpar))
}

hchange=function(par,lower,upper,dist,round=TRUE,...)
{ 
    D=length(par)
    step=dist(D,...)
    if(round) 
      step=round(step)
    par1=par+step
    return(ifelse(par1<lower,lower,ifelse(par1>upper,upper,par1)))
}

ichange=function(par,lower,upper)
{ 
    hchange(par,lower,upper,rnorm,mean=0,sd=1) 
}

# Функция для вызова определённого метода

In [14]:
%%R

result=function(Mymethod, Myfunction, arg)
{ 
    answer = "Такого метода нет"
    time = Sys.time()
    if(Mymethod == "Отжиг"){
        C=list(maxit=10,temp=10,tmax=1,trace=TRUE,REPORT=1)
        answer=optim(arg,Myfunction,gr=bchange,method="SANN", control=C)
        cat("best:",answer$par,"f:",answer$value,"(max: fs:",sum(answer$par),")\n")
    }
    if(Mymethod == "МК"){
        answer=mcsearch(10000,arg,rep(0,5),Myfunction,"min")
        cat("best:",answer$sol,"f:",answer$eval,"\n")
    }
    if(Mymethod == "Вершина"){
        answer=list(maxit=10,REPORT=1)
        D=8
        hclimbing(arg,Myfunction,change=ichange,lower=rep(0,D),upper=rep(1,D), control=answer,type="min")
    }
    if(answer == "Такого метода нет"){
        cat(answer)
        return (-1)
    }
    time = paste0(round(as.numeric(difftime(time1 = Sys.time(), time2 = time, units = "secs")), 3))
    cat("Метод:", Mymethod, "\nвремя работы алгоритма:",time)
    cat("\n")
    return (list(timer = as.numeric(time), method = Mymethod))
}

# Примеры применения методов к двум целевым функциям

In [4]:
%%R

arg1= c(0, 0)
func1=function(x) {
    sum(x**2-2*x+30) 
}
D = 3
arg2= rep(0, D)
func2=function(x) {
    sum(exp(x+3)+x**3) 
}

In [15]:
%%R
s1_1 = result("Отжиг", func1, arg1)
s1_2 = result("Отжиг", func2, arg2)

sann objective function values
initial       value 60.000000
iter        1 value 59.000000
iter        2 value 59.000000
iter        3 value 58.000000
iter        4 value 58.000000
iter        5 value 58.000000
iter        6 value 58.000000
iter        7 value 58.000000
iter        8 value 58.000000
iter        9 value 58.000000
final         value 58.000000
sann stopped after 9 iterations
best: 1 1 f: 58 (max: fs: 2 )
Метод: Отжиг 
время работы алгоритма: 0.037
sann objective function values
initial       value 60.256611
iter        1 value 60.256611
iter        2 value 60.256611
iter        3 value 60.256611
iter        4 value 60.256611
iter        5 value 60.256611
iter        6 value 60.256611
iter        7 value 60.256611
iter        8 value 60.256611
iter        9 value 60.256611
final         value 60.256611
sann stopped after 9 iterations
best: 0 0 0 f: 60.25661 (max: fs: 0 )
Метод: Отжиг 
время работы алгоритма: 0.036


In [16]:
%%R
s2_1 = result("МК", func1, arg1)
s2_2 = result("МК", func2, arg2)

best: 0 0 f: 60 
Метод: МК 
время работы алгоритма: 0.096
best: 0 0 0 f: 60.25661 
Метод: МК 
время работы алгоритма: 0.115


In [17]:
%%R
s3_1 = result("Вершина", func1, arg1)
s3_2 = result("Вершина", func2, arg2)

i: 1 s: 0 0 f: 60 s’ 1 0 1 0 1 0 1 0 f: 236 
i: 2 s: 0 0 f: 60 s’ 1 0 1 0 1 0 1 0 f: 236 
i: 3 s: 0 0 f: 60 s’ 0 1 0 1 0 1 0 1 f: 236 
i: 4 s: 0 0 f: 60 s’ 1 0 1 0 1 0 1 0 f: 236 
i: 5 s: 0 0 f: 60 s’ 0 1 0 1 0 1 0 1 f: 236 
i: 6 s: 0 0 f: 60 s’ 0 1 0 1 0 1 0 1 f: 236 
i: 7 s: 0 0 f: 60 s’ 0 0 0 0 0 0 0 0 f: 240 
i: 8 s: 0 0 f: 60 s’ 1 0 1 0 1 0 1 0 f: 236 
i: 9 s: 0 0 f: 60 s’ 0 0 0 0 0 0 0 0 f: 240 
i: 10 s: 0 0 f: 60 s’ 0 0 0 0 0 0 0 0 f: 240 
best: 0 0 f: 60 
Метод: Вершина 
время работы алгоритма: 0.656
i: 1 s: 0 0 0 f: 60.25661 s’ 0 1 0 0 1 0 0 1 f: 267.2221 
i: 2 s: 0 0 0 f: 60.25661 s’ 0 0 0 0 0 0 0 0 f: 160.6843 
i: 3 s: 0 0 0 f: 60.25661 s’ 0 1 0 0 1 0 0 1 f: 267.2221 
i: 4 s: 0 0 0 f: 60.25661 s’ 0 1 0 0 1 0 0 1 f: 267.2221 
i: 5 s: 0 0 0 f: 60.25661 s’ 1 1 0 1 1 0 1 1 f: 373.76 
i: 6 s: 0 0 0 f: 60.25661 s’ 1 1 0 1 1 0 1 1 f: 373.76 
i: 7 s: 0 0 0 f: 60.25661 s’ 1 0 1 1 0 1 1 0 f: 338.2474 
i: 8 s: 0 0 0 f: 60.25661 s’ 0 1 1 0 1 1 0 1 f: 338.2474 
i: 9 s: 0 0 0 f: 60.25661 

# Вывод для целевых функций методов, отработавших за наименьшее время

In [18]:
%%R
arr1 = data.frame(timer = c(s1_1$timer, s2_1$timer, s3_1$timer), method = c(s1_1$method, s2_1$method, s3_1$method))
arr2 = data.frame(timer = c(s1_2$timer, s2_2$timer, s3_2$timer), method = c(s1_2$method, s2_2$method, s3_2$method))
index_arr_sort_1 = order(arr1$timer)
index_arr_sort_2 = order(arr2$timer)
print(arr1[index_arr_sort_1[1],])
print(arr2[index_arr_sort_2[1],])

  timer method
1 0.037  Отжиг
  timer method
1 0.036  Отжиг
